Movie data was provided in the project description (project 3)<br>
This notebook focuses on the problem of predicting whether or not a movie has a character in it who uses fire

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import random


from transformers import BertTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

movies = pd.read_json('IMDB_movie_details.json',lines=True)
movies

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...
...,...,...,...,...,...,...,...
1567,tt0289879,Evan Treborn grows up in a small town with his...,1h 53min,"[Sci-Fi, Thriller]",7.7,2004-01-23,"In the year 1998, Evan Treborn (Ashton Kutcher..."
1568,tt1723811,Brandon is a 30-something man living in New Yo...,1h 41min,[Drama],7.2,2012-01-13,"Brandon (Michael Fassbender) is a successful, ..."
1569,tt5013056,Evacuation of Allied soldiers from the British...,1h 46min,"[Action, Drama, History]",8.1,2017-07-21,The film alternates between three different pe...
1570,tt0104014/,"For a while now, beautiful 24-year-old Diana B...",1h 33min,"[Comedy, Drama]",5.3,1992-02-21,


One way to do this is to compare movies to anime, where many anime have fire users<br>
Anime data was from Kaggle, scrapped from MyAnimeList https://www.kaggle.com/datasets/marlesson/myanimelist-dataset-animes-profiles-reviews

In [2]:
anime = pd.read_csv('animes.csv')
anime = anime.drop_duplicates(subset=['title'])
anime

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...
...,...,...,...,...,...,...,...,...,...,...,...,...
19002,10075,Naruto x UT,All-new animation offered throughout UNIQLO cl...,"['Action', 'Comedy', 'Super Power', 'Martial A...","Jan 1, 2011",1.0,34155,2382,1728.0,7.50,https://cdn.myanimelist.net/images/anime/3/304...,https://myanimelist.net/anime/10075/Naruto_x_UT
19003,35828,Miira no Kaikata,High school student Sora Kashiwagi is accustom...,"['Slice of Life', 'Comedy', 'Supernatural']","Jan 12, 2018 to Mar 30, 2018",12.0,61459,1648,1727.0,7.50,https://cdn.myanimelist.net/images/anime/1486/...,https://myanimelist.net/anime/35828/Miira_no_K...
19004,10378,Shinryaku!? Ika Musume,"After regaining her squid-like abilities, Ika ...","['Slice of Life', 'Comedy', 'Shounen']","Sep 27, 2011 to Dec 25, 2011",12.0,67422,1547,1548.0,7.56,https://cdn.myanimelist.net/images/anime/6/301...,https://myanimelist.net/anime/10378/Shinryaku_...
19005,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,https://cdn.myanimelist.net/images/anime/12/79...,https://myanimelist.net/anime/33082/Kingsglaiv...


Information on anime characters was from MyAnimeList, saved as HTML webpages<br>
i.e. MyAnimeList page for Natsu Dragneel: https://myanimelist.net/character/5187/Natsu_Dragneel<br>
Compiled a sample of characters who use fire and sample of characters from anime where no fire user is present. The prefix `(MC)` indicates fire users who are the main character of their respective show (i.e. Natsu Dragneel is the main character in Fairy Tail)<br>
This had to be handled manually because it's difficult to deterministically automate whether or not each anime has a fire user in it for 16214 anime<br>

In [3]:
import os
from html import unescape

substrcheck = np.vectorize(lambda a,A: a in A,otypes=[bool])

def character_shows(character_html):
    Shows = np.array(character_html.split('</a>'))
    Shows = np.vectorize(lambda a: a.split(">")[-1],otypes=[str])(Shows[np.where(substrcheck('<td valign="top" class="borderClass">',Shows))])
    return np.unique(Shows)

def apply_to(thing,folder,D,fun):
    try:
        with open(folder+"/"+thing,"r",encoding='utf-8') as f:
            D[thing.split(".")[0]] = fun(unescape(f.read()))
    except PermissionError:
        pass

fire_shows = {}
firent_shows = {}

np.vectorize(apply_to,otypes=[None])(np.array(os.listdir('fire users')),'fire users',fire_shows,character_shows)
np.vectorize(apply_to,otypes=[None])(np.array(os.listdir('firent users')),'firent users',firent_shows,character_shows)
fire_shows

{'(MC) Agumon': array(['Digimon Adventure', 'Digimon Adventure 02',
        'Digimon Adventure 02 Movies',
        'Digimon Adventure 02: Diablomon no Gyakushuu',
        'Digimon Adventure 3D: Digimon Grand Prix!',
        'Digimon Adventure Movie', 'Digimon Adventure tri. 1: Saikai',
        'Digimon Adventure tri. 2: Ketsui',
        'Digimon Adventure tri. 3: Kokuhaku',
        'Digimon Adventure tri. 4: Soushitsu',
        'Digimon Adventure tri. 5: Kyousei',
        'Digimon Adventure tri. 6: Bokura no Mirai', 'Digimon Adventure:',
        'Digimon Adventure: 20-shuunen Memorial Story',
        'Digimon Adventure: Bokura no War Game!',
        'Digimon Adventure: Last Evolution Kizuna',
        'Digimon Adventure: V-Tamer 01',
        'Digimon Adventure: V-Tamer 01 - Arata naru Yuuki',
        'Digimon D-Cyber', 'Digimon Next', 'Digimon Savers',
        'Digimon Savers 3D: Digital World Kiki Ippatsu!',
        'Digimon Savers the Movie: Kyuukyoku Power! Burst Mode Hatsudou!!',
  

In [4]:
fire_mc = np.array([])
fire_mcnt = np.array([])
firent = np.array([])

for show in fire_shows.items():
    if "(MC)" in show[0]:
        fire_mc = np.concatenate((fire_mc,show[1]))
    else:
        fire_mcnt = np.concatenate((fire_mc,show[1]))

for show in firent_shows.items():
    firent = np.concatenate((firent,show[1]))

fire_mc = np.unique(fire_mc)
fire_mcnt = np.unique(fire_mcnt)
firent = np.unique(firent)
fire_mc

array(['A Ripple in Time', 'Ao no Exorcist', 'Ao no Exorcist Movie',
       'Ao no Exorcist Movie Special', 'Ao no Exorcist: Kuro no Iede',
       'Ao no Exorcist: Kyoto Fujouou-hen',
       'Ao no Exorcist: Kyoto Fujouou-hen OVA',
       'Ao no Exorcist: Shimane Illuminati-hen', 'Ao no Exorcist: Ura Ex',
       'Ao no Exorcist: Yuki no Hate-hen', 'Bakugan Battle Brawlers',
       'Bakugan Battle Brawlers: Gundalian Invaders',
       'Bakugan Battle Brawlers: Mechtanium Surge',
       'Bakugan Battle Brawlers: New Vestroia', 'Beyblade Burst',
       'Beyblade Burst Chouzetsu', 'Beyblade Burst Gachi',
       'Beyblade Burst Sparking', 'CoroCoro Monster Ball',
       'Dengeki Bunko 2007 Movie Festival Special', 'Dengeki! Pikachu',
       'Digimon Adventure', 'Digimon Adventure 02',
       'Digimon Adventure 02 Movies',
       'Digimon Adventure 02: Diablomon no Gyakushuu',
       'Digimon Adventure 3D: Digimon Grand Prix!',
       'Digimon Adventure Movie', 'Digimon Adventure tri. 1: Sai

For the synopsis of each anime, remove stop words and punctuation

In [5]:
fire_mc_synopsis_full = {}
fire_mcnt_synopsis_full = {}
firent_synopsis = {}

stopcheck = np.vectorize(lambda w: w in stopwords.words('english'),otypes=[bool])

def fill_synopsis(title,D,label):
    try:
        D[title] = np.array(np.array(anime[anime['title']==title]['synopsis'])[0].replace('(Source:','[Written by MAL Rewrite]').split('[Written by MAL Rewrite]')[0].replace('\r\n',' ').replace('.','').replace(',','').replace('!','').replace('"','').replace('(','').replace(')','').lower().split(' '))
        D[title] = np.delete(D[title],np.where(D[title]==''))
        D[title] = np.delete(D[title],np.where(stopcheck(D[title])))
        D[title] = ' '.join(D[title])
        while '  ' in D[title]:
            D[title] = D[title].replace('  ',' ')
        D[title] = [D[title],label]
    except:
        pass

np.vectorize(lambda p: fill_synopsis(p,fire_mc_synopsis_full,1))(fire_mc)
np.vectorize(lambda p: fill_synopsis(p,fire_mcnt_synopsis_full,1))(fire_mcnt)
np.vectorize(lambda p: fill_synopsis(p,firent_synopsis,0))(firent)
fire_mc_synopsis_full

{'Ao no Exorcist': ['humans demons two sides coin assiah gehenna respective worlds way travel realms means possession like ghost stories however satan ruler gehenna cannot find suitable host possess therefore remains imprisoned world desperate attempt conquer assiah sends son instead intending eventually grow vessel capable possession demon king ao exorcist follows rin okumura appears ordinary somewhat troublesome teenager—that one day ambushed demons world turns upside discovers fact son satan demon father wishes return conquer assiah together wanting join king gehenna rin decides begin training become exorcist fight defend assiah alongside brother yukio',
  1],
 'Ao no Exorcist Movie': ['atmosphere true cross academy lively boisterous days leading grand festival held every 11 years time okumura rin entrusted responsibility suppressing berserk phantom train midst mission meets devil whose appearance young boy',
  1],
 'Ao no Exorcist Movie Special': ['collaboration blue exorcist kakko

In [6]:
fire_mc_synopsis = {}
fire_mcnt_synopsis = {}

# reducing sample size
for k in random.sample(list(fire_mc_synopsis_full.keys()),60):
    fire_mc_synopsis[k] = fire_mc_synopsis_full[k]
for k in random.sample(list(fire_mcnt_synopsis_full.keys()),60):
    fire_mcnt_synopsis[k] = fire_mcnt_synopsis_full[k]

fire_mc_synopsis

{'Pokemon Advanced Generation': ["participating johto league satoshi decides begin anew yet another journey; time pikachu land houen though alone long 10-year-old haruka dislikes pokemon wants come along see world travels pikachu destroys bicycle satoshi also meets old companion takeshi haruka's younger brother masato together travel along houen battling gym leaders entering contests events way houen league",
  1],
 'Ao no Exorcist: Kuro no Iede': ['feeling slighted rin kuro goes hunt new master find anyone truly replace rin?',
  1],
 'Pokemon: The Origin': ["pokémon marvelous creatures come variety types sizes abilities powers personalities diverse numerous doctor yukinari ookido dedicated life studying fascinating beings caught trained traded battled there's much lab though mind ookido entrusts two young boys pokémon computerized encyclopedia catalog one green brash passionate slightly arrogant doctor ookido's nephew boy red equally passionate filled wide-eyed mildly naive sense wond

In [7]:
anime_fire = pd.DataFrame.from_dict({**fire_mc_synopsis, **fire_mcnt_synopsis, **firent_synopsis},orient='index')
anime_fire.rename(columns={0: 'synopsis',1: 'has_fire'}, inplace=True)
anime_fire.drop(anime_fire[anime_fire['synopsis'] == ''].index, inplace=True)
anime_fire

,synopsis,has_fire
Pokemon Advanced Generation,participating johto league satoshi decides beg...,1
Ao no Exorcist: Kuro no Iede,feeling slighted rin kuro goes hunt new master...,1
Pokemon: The Origin,pokémon marvelous creatures come variety types...,1
Pokemon Best Wishes! Season 2: Decolora Adventure,encounter team plasma satoshi iris dent start ...,1
Digimon Adventure tri. 4: Soushitsu,first meicoomon's rampage reboot ended happeni...,1
...,...,...
Hori-san to Miyamura-kun,within everyone exists side preferably kept hi...,0
Ouran Koukou Host Club,haruhi fujioka bright scholarship candidate ra...,0
Shingeki no Kyojin,centuries ago mankind slaughtered near extinct...,0
Shingeki no Kyojin Season 2,centuries humanity hunted giant mysterious pre...,0


In [8]:
N = 200
moviesample = movies.sample(n=N)

def cleanup(s):
    S = np.array(s.replace('\r\n',' ').replace('.','').replace(',','').replace('!','').replace('"','').replace('(','').replace(')','').lower().split(' '))
    S = np.delete(S,np.where(S==''))
    S = np.delete(S,np.where(stopcheck(S)))
    S = ' '.join(S)
    while '  ' in S:
        S = S.replace('  ',' ')
    return S
cleanup = np.vectorize(cleanup,otypes=[str])

moviesample.loc[:,'plot_synopsis'] = cleanup(moviesample.loc[:,'plot_synopsis'])
moviesample['has_fire'] = 0 # this is technically incorrect but we'll check it out later
moviesample.to_csv('movierngsample{}.csv'.format(N),index=False)
moviesample.drop(moviesample[moviesample['plot_synopsis'] == ''].index, inplace=True)
moviesample

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis,has_fire
1252,tt2872732,It was supposed to be a simple job. All Lucy h...,1h 29min,"[Action, Sci-Fi, Thriller]",6.4,2014-07-25,opening shot see cell split multiple cells see...,0
247,tt0099653,"Sam Wheat is a banker, Molly Jensen is an arti...",2h 7min,"[Drama, Fantasy, Romance]",7.0,1990-07-13,sam wheat patrick swayze banker molly jensen d...,0
28,tt0017136,"Sometime in the future, the city of Metropolis...",2h 33min,"[Drama, Sci-Fi]",8.3,1927-03-13,film set year 2026 extraordinary gothic skyscr...,0
1296,tt0120912,"Kay and Jay reunite to provide our best, last ...",1h 28min,"[Action, Adventure, Comedy]",6.1,2002-07-03,columbia torch buzzes flashes like neuralizera...,0
56,tt0406375,Danny and Walter are two brothers who barely g...,1h 41min,"[Action, Adventure, Comedy]",6.1,2005-11-11,two boys found game play started play game sta...,0
...,...,...,...,...,...,...,...,...
950,tt0101507,John Singleton's portrayal of social problems ...,1h 52min,"[Crime, Drama]",7.8,1991-07-12,story opens 1984 focusing three young black ma...,0
1529,tt1375666,"Dom Cobb is a skilled thief, the absolute best...",2h 28min,"[Action, Adventure, Sci-Fi]",8.8,2010-07-16,young man exhausted delirious washes beach loo...,0
1498,tt1014759,"Alice, an unpretentious and individual 19-year...",1h 48min,"[Adventure, Family, Fantasy]",6.5,2010-03-05,alice kingsleigh mia wasikowska daughter charl...,0
130,tt1663662,"When monstrous creatures, known as Kaiju, star...",2h 11min,"[Action, Adventure, Sci-Fi]",7.0,2013-07-12,film begins 2013 deep beneath pacific ocean fi...,0


Compile all synopsis in the same corpus to ensure a tf-idf vectorization that can be uniformly applied to all entries

In [9]:
all_synopsis = np.concatenate((np.array([f[0] for f in {**fire_mc_synopsis, **fire_mcnt_synopsis, **firent_synopsis}.values()]),np.array(moviesample['plot_synopsis'])))
all_synopsis

array(["participating johto league satoshi decides begin anew yet another journey; time pikachu land houen though alone long 10-year-old haruka dislikes pokemon wants come along see world travels pikachu destroys bicycle satoshi also meets old companion takeshi haruka's younger brother masato together travel along houen battling gym leaders entering contests events way houen league",
       'feeling slighted rin kuro goes hunt new master find anyone truly replace rin?',
       "pokémon marvelous creatures come variety types sizes abilities powers personalities diverse numerous doctor yukinari ookido dedicated life studying fascinating beings caught trained traded battled there's much lab though mind ookido entrusts two young boys pokémon computerized encyclopedia catalog one green brash passionate slightly arrogant doctor ookido's nephew boy red equally passionate filled wide-eyed mildly naive sense wonder pokémon: origin follows red journey region kanto attempt complete pokédex captur

In [10]:
tfidf = TfidfVectorizer(stop_words=stopwords.words('english'))

X = tfidf.fit_transform(all_synopsis)
X
X_anime = X[:anime_fire.shape[0],:]
y_anime = np.array(anime_fire['has_fire'])
X_movies = X[anime_fire.shape[0]:,:]

Try classifying via SVM

In [11]:
from sklearn.svm import SVC

svm_fire = SVC()
svm_fire.fit(X_anime,y_anime)
y_movies_svm = svm_fire.predict(X_movies)
y_movies_svm

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

Try Decision Trees

In [12]:
from sklearn import tree

tree_fire = tree.DecisionTreeClassifier()
tree_fire.fit(X_anime,y_anime)
y_movies_tree = tree_fire.predict(X_movies)
y_movies_tree

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int64)

pain

In [13]:
# code copied from Chi

class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_dim, n_layers, bidirectional, dropout):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        lstm_out, _ = self.lstm(embedded)
        output = self.fc(lstm_out[:, -1, :])
        return output
    

# Training function
def train_lstm(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    total_acc = 0

    # Wrap the dataloader with tqdm for progress tracking
    for batch in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        labels = batch['label'].to(device)

        # Forward pass
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        preds = torch.argmax(outputs, dim=1)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_acc += accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())

    return total_loss / len(dataloader), total_acc / len(dataloader)


# Evaluation function
def evaluate_lstm(model, dataloader):
    model.eval()
    total_acc = 0

    with torch.no_grad():
        # Wrap the dataloader with tqdm for progress tracking
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids)
            preds = torch.argmax(outputs, dim=1)

            total_acc += accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())

    return total_acc / len(dataloader)

In [14]:
# code copied from Chi

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class FireDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = tokenizer.encode_plus(
            text,
            max_length=5000,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

train_ds = FireDataset(anime_fire['synopsis'].tolist(),anime_fire['has_fire'].tolist())

In [15]:
test_ds = FireDataset(moviesample['plot_synopsis'].tolist(),moviesample['has_fire'].tolist())

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=8, shuffle=False)

embedding_dim = 100
hidden_dim = 128
vocab_size = tokenizer.vocab_size
output_dim = 2
n_layers = 3
bidirectional = True
dropout = 0.5

lstm_model = LSTMClassifier(embedding_dim, hidden_dim, vocab_size, output_dim, n_layers, bidirectional, dropout)
lstm_model = lstm_model.to(device)
optimizer = Adam(lstm_model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [16]:
train_losses = []
train_accuracies = []

n_epochs = 5
# Initialize tqdm progress bar
for epoch in tqdm(range(n_epochs), desc='Training Epochs'):
    train_loss, train_acc = train_lstm(lstm_model, train_loader, optimizer, criterion)
    
    # Append metrics to lists
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    
    # Print epoch information
    tqdm.write(f'Epoch {epoch+1}: Train Loss {train_loss:.4f}, Train Acc {train_acc:.4f}')
    
    # Save model checkpoint every 10 epochs
    if (epoch + 1) % 10 == 0:
        checkpoint_path = f'lstm_model_epoch_{epoch+1}.pth'
        torch.save(lstm_model.state_dict(), checkpoint_path)
        print(f'Model checkpoint saved to {checkpoint_path}')

# Evaluate on test set
test_acc = evaluate_lstm(lstm_model, test_loader)
print(f'Test Accuracy: {test_acc:.4f}')

Training Epochs:  20%|██        | 1/5 [06:40<26:41, 400.25s/it]

Epoch 1: Train Loss 0.6680, Train Acc 0.6306



Training Epochs:  40%|████      | 2/5 [13:10<19:43, 394.56s/it]

Epoch 2: Train Loss 0.6420, Train Acc 0.6611



Training Epochs:  60%|██████    | 3/5 [19:50<13:13, 396.68s/it]

...

Epoch 4: Train Loss 0.6361, Train Acc 0.6639



Training Epochs: 100%|██████████| 5/5 [33:43<00:00, 404.69s/it]


Epoch 5: Train Loss 0.6525, Train Acc 0.6611


Evaluating: 100%|██████████| 22/22 [02:58<00:00,  8.11s/it]

Test Accuracy: 0.0000


Predicts everything as having a fire user even though most of them probably don't...<br>
Maybe plot synopsis isn't a good indicator on whether or not a movie has a fire user in it..?<br>

Ok now let's focus only on the anime with fire users and assess how similar the movie descriptioins are to the fire anime descriptions. We will then sort the movies based on the highest similarity across all the fire anime, then that'll give us somewhat of an idea on what movies are likely to have fire users

In [18]:
ANIMESYNOPSIS,MOVIESYNOPSIS = np.meshgrid(np.concatenate((np.array(list(fire_mc_synopsis_full.values()))[:,0],np.array(list(fire_mcnt_synopsis_full.values()))[:,0])),moviesample['plot_synopsis'])

movies_by_score = lambda scores: np.array(sorted(zip(moviesample['movie_id'],scores),key=lambda x: x[1],reverse=True))

In [19]:
from nltk.metrics.distance import jaccard_distance

jaccard_score = np.vectorize(lambda a,m:1-jaccard_distance(set(a),set(m)),otypes=[float])

J = jaccard_score(ANIMESYNOPSIS,MOVIESYNOPSIS)
movies_scored_J = movies_by_score(J@np.ones(J.shape[1]))
movies_scored_J

array([['tt0343660', '138.19336957345996'],
       ['tt0118694', '135.032949409923'],
       ['tt2293640', '134.97817863957528'],
       ['tt0118929', '134.7467049657359'],
       ['tt0322259', '134.30716021366578'],
       ['tt3235888', '134.30716021366578'],
       ['tt1109624', '134.30716021366578'],
       ['tt0133751', '134.30716021366578'],
       ['tt1229238', '134.30716021366578'],
       ['tt0106856', '134.30716021366578'],
       ['tt0103060', '134.07373951527217'],
       ['tt0108358', '133.7612053118907'],
       ['tt0312528', '133.33961106921342'],
       ['tt0099938', '132.99139341296228'],
       ['tt0406375', '132.34584863494766'],
       ['tt0099253', '132.17489854471467'],
       ['tt1032755', '131.83836107756883'],
       ['tt1631867', '131.83836107756883'],
       ['tt2250912', '131.83836107756883'],
       ['tt0307987', '131.0068920054623'],
       ['tt0492956', '130.41703641867304'],
       ['tt2338151', '130.32841372922917'],
       ['tt0101545', '130.32841372922

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cosine_score(a,m): # Adapted from Yutong's code
    cv = CountVectorizer()
    AM = cv.fit_transform([a,m]).toarray()
    return cosine_similarity(AM)[0,1]
cosine_score = np.vectorize(cosine_score,otypes=[float])

C = cosine_score(ANIMESYNOPSIS,MOVIESYNOPSIS)
movies_scored_C = movies_by_score(C@np.ones(C.shape[1]))
movies_scored_C

array([['tt0478304', '9.216575550378383'],
       ['tt0398808', '8.687605248366822'],
       ['tt0120655', '6.889405920820177'],
       ['tt0311429', '6.4741977908740385'],
       ['tt2872732', '6.471395552882056'],
       ['tt0371724', '6.4584377702729725'],
       ['tt0238380', '6.441394146610345'],
       ['tt0360717', '6.4265189351229735'],
       ['tt0107048', '6.370960877782074'],
       ['tt0314331', '6.2278112194152815'],
       ['tt0078748', '6.196082845198438'],
       ['tt5311514', '6.186755937815668'],
       ['tt0101507', '6.046696649528513'],
       ['tt1632708', '5.969755276503545'],
       ['tt0104694', '5.91250387343139'],
       ['tt1483013', '5.874901531135253'],
       ['tt1596363', '5.834865431398727'],
       ['tt0976051', '5.829243452160289'],
       ['tt0115639', '5.804534261476145'],
       ['tt1663662', '5.792271693959215'],
       ['tt0117802', '5.711981927611594'],
       ['tt0408306', '5.633659368479006'],
       ['tt0108037', '5.521174559024544'],
       [

Neither of the first movies have to do with fire...<br>
Ok what if we filter movies by genre

In [21]:
genres = np.unique(np.concatenate(movies['genre']))
genres

array(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror',
       'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport',
       'Thriller', 'War', 'Western'], dtype='<U9')

Action, Adventure, and Fantasy seem like good target genres

In [22]:
firedupgenres = lambda b: np.any(np.vectorize(lambda p: p in b,otypes=[bool])(['Action','Adventure','Fantasy']))

pog_movies = movies[movies['genre'].apply(firedupgenres)]
pog_movies

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
5,tt0286716,"Bruce Banner, a brilliant scientist with a clo...",2h 18min,"[Action, Sci-Fi]",5.7,2003-06-20,Bruce Banner (Eric Bana) is a research scienti...
6,tt0090605,57 years after Ellen Ripley had a close encoun...,2h 17min,"[Action, Adventure, Sci-Fi]",8.4,1986-07-18,"After the opening credits, we see a spacecraft..."
8,tt0121765,"Ten years after the invasion of Naboo, the Gal...",2h 22min,"[Action, Adventure, Fantasy]",6.6,2002-05-16,The opening crawl reveals that the Galactic Re...
...,...,...,...,...,...,...,...
1559,tt0295297,Forced to spend his summer holidays with his m...,2h 41min,"[Adventure, Family, Fantasy]",7.4,2002-11-15,"While home with the Dursleys for the summer, H..."
1561,tt1323594,In a happy suburban neighborhood surrounded by...,1h 35min,"[Animation, Adventure, Comedy]",7.7,2010-07-09,The film begins with the news revealing that a...
1562,tt0120891,Jim West is a guns-a-blazing former Civil War ...,1h 46min,"[Action, Comedy, Sci-Fi]",4.8,1999-06-30,The story opens in Louisiana in 1869. A man fi...
1563,tt0120655,An abortion clinic worker with a special herit...,2h 10min,"[Adventure, Comedy, Drama]",7.3,1999-11-12,The film opens with a homeless man (Bud Cort) ...


In [23]:
pogmoviesample = pog_movies.sample(n=N)

pogmoviesample.loc[:,'plot_synopsis'] = cleanup(pogmoviesample.loc[:,'plot_synopsis'])
pogmoviesample['has_fire'] = 0 # this is technically incorrect but we'll check it out later
pogmoviesample.to_csv('movierngsample{}.csv'.format(N),index=False)
pogmoviesample.drop(pogmoviesample[pogmoviesample['plot_synopsis'] == ''].index, inplace=True)
pogmoviesample

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis,has_fire
1319,tt0796366,"On the day of James Kirk's birth, his father d...",2h 7min,"[Action, Adventure, Sci-Fi]",8.0,2009-05-08,year 2233 federation starship uss kelvin inves...,0
293,tt1637688,Welcome to a world where time has become the u...,1h 49min,"[Action, Sci-Fi, Thriller]",6.7,2011-10-28,voiceover describes state world place time bec...,0
604,tt0120685,Following the French atomic bomb tests in the ...,2h 19min,"[Action, Sci-Fi, Thriller]",5.3,1998-05-20,movie starts reel showing nuclear tests island...,0
1111,tt0017925,"Johnnie loves his train (""The General"") and An...",1h 7min,"[Action, Adventure, Comedy]",8.2,1927-02-24,western & atlantic flyer speeds marietta georg...,0
774,tt0245429,Chihiro and her parents are moving to a small ...,2h 5min,"[Animation, Adventure, Family]",8.6,2003-03-28,&#12302;&#21315;&#12392;&#21315;&#23563;&#1239...,0
...,...,...,...,...,...,...,...,...
1287,tt0120660,Robert Dean is a mild-mannered lawyer who work...,2h 12min,"[Action, Crime, Mystery]",7.3,1998-11-20,congress close passing legislation expand surv...,0
1122,tt0942385,Action movie star Tugg Speedman is on the down...,1h 47min,"[Action, Comedy]",7.0,2008-08-13,commercial depicts rapper alpa chino brandon j...,0
348,tt2527336,Rey develops her newly discovered abilities wi...,2h 32min,"[Action, Adventure, Fantasy]",7.5,2017-12-15,opening crawl latest film 'star wars episode v...,0
20,tt0120783,When two pre-teens named Hallie and Annie meet...,2h 8min,"[Adventure, Comedy, Drama]",6.5,1998-07-29,1986 nick parker dennis quaid elizabeth james ...,0


In [24]:
all_synopsis = np.concatenate((np.array([f[0] for f in {**fire_mc_synopsis, **fire_mcnt_synopsis, **firent_synopsis}.values()]),np.array(pogmoviesample['plot_synopsis'])))
all_synopsis

array(["participating johto league satoshi decides begin anew yet another journey; time pikachu land houen though alone long 10-year-old haruka dislikes pokemon wants come along see world travels pikachu destroys bicycle satoshi also meets old companion takeshi haruka's younger brother masato together travel along houen battling gym leaders entering contests events way houen league",
       'feeling slighted rin kuro goes hunt new master find anyone truly replace rin?',
       "pokémon marvelous creatures come variety types sizes abilities powers personalities diverse numerous doctor yukinari ookido dedicated life studying fascinating beings caught trained traded battled there's much lab though mind ookido entrusts two young boys pokémon computerized encyclopedia catalog one green brash passionate slightly arrogant doctor ookido's nephew boy red equally passionate filled wide-eyed mildly naive sense wonder pokémon: origin follows red journey region kanto attempt complete pokédex captur

In [25]:
tfidf = TfidfVectorizer(stop_words=stopwords.words('english'))

X = tfidf.fit_transform(all_synopsis)
X
X_anime = X[:anime_fire.shape[0],:]
y_anime = np.array(anime_fire['has_fire'])
X_movies = X[anime_fire.shape[0]:,:]

In [26]:
from sklearn.svm import SVC

svm_fire = SVC()
svm_fire.fit(X_anime,y_anime)
y_movies_svm = svm_fire.predict(X_movies)
y_movies_svm

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64)

In [27]:
from sklearn import tree

tree_fire = tree.DecisionTreeClassifier()
tree_fire.fit(X_anime,y_anime)
y_movies_tree = tree_fire.predict(X_movies)
y_movies_tree

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
      dtype=int64)

Yeah nothing's changed, but let's hope we actually get something meaningful from similarity ranking?

In [34]:
ANIMESYNOPSIS,POGMOVIESYNOPSIS = np.meshgrid(np.concatenate((np.array(list(fire_mc_synopsis_full.values()))[:,0],np.array(list(fire_mcnt_synopsis_full.values()))[:,0])),pogmoviesample['plot_synopsis'])
pogmovies_by_score = lambda scores: np.array(sorted(zip(pogmoviesample['movie_id'],scores),key=lambda x: x[1],reverse=True))

In [35]:
J = jaccard_score(ANIMESYNOPSIS,POGMOVIESYNOPSIS)
movies_scored_J = pogmovies_by_score(J@np.ones(J.shape[1]))
movies_scored_J

array([['tt0113957', '137.76988929297744'],
       ['tt0112442', '136.96716522935517'],
       ['tt0149261', '134.30716021366578'],
       ['tt1109624', '134.30716021366578'],
       ['tt2404463', '134.30716021366578'],
       ['tt1229238', '134.30716021366578'],
       ['tt0198781', '134.30716021366578'],
       ['tt0052618', '134.30716021366578'],
       ['tt0338348', '133.96439959389556'],
       ['tt0312528', '133.33961106921342'],
       ['tt0120762', '131.83836107756883'],
       ['tt1187064', '131.83836107756883'],
       ['tt1509767', '131.83836107756883'],
       ['tt1156398', '131.83836107756883'],
       ['tt0245844', '131.83836107756883'],
       ['tt0120660', '131.26540503298094'],
       ['tt2193021', '130.67992893031416'],
       ['tt0110357', '130.66109440843786'],
       ['tt0325980', '130.66109440843786'],
       ['tt0120855', '130.66109440843786'],
       ['tt0948470', '130.66109440843786'],
       ['tt0417741', '130.66109440843786'],
       ['tt0230030', '130.472989

In [36]:
C = cosine_score(ANIMESYNOPSIS,POGMOVIESYNOPSIS)
movies_scored_C = pogmovies_by_score(C@np.ones(C.shape[1]))
movies_scored_C

array([['tt1637688', '8.881458376549183'],
       ['tt1211837', '8.688751391258098'],
       ['tt0372588', '7.664652097365204'],
       ['tt0106611', '7.429587023772696'],
       ['tt0770828', '6.724537600046969'],
       ['tt0980970', '6.665074083536003'],
       ['tt0376994', '6.655834980366053'],
       ['tt0112384', '6.588403620673604'],
       ['tt0177789', '6.5064526512669785'],
       ['tt0371724', '6.4584377702729725'],
       ['tt0107048', '6.370960877782074'],
       ['tt0257076', '6.262393252377596'],
       ['tt0499448', '6.050122802973868'],
       ['tt0120685', '6.016792409993099'],
       ['tt0800080', '5.932571227283309'],
       ['tt0413267', '5.921012124965228'],
       ['tt1483013', '5.874901531135253'],
       ['tt2527336', '5.777502231108275'],
       ['tt2109248', '5.702701843749059'],
       ['tt0938283', '5.685508509396767'],
       ['tt0076759', '5.675589284590705'],
       ['tt1060277', '5.568673399438758'],
       ['tt1253863', '5.526740302567624'],
       ['

Firen't...

Ok how about cross referencing all descriptions and penalizing correlation with anime that haven't fire?

In [37]:
ALLANIMESYNOPSIS,POGMOVIESYNOPSIS = np.meshgrid(anime_fire['synopsis'],pogmoviesample['plot_synopsis'])

score_weights = np.array(anime_fire['has_fire'])
score_weights[np.where(score_weights==0)] = -1
score_weights

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
      dtype=int64)

In [38]:
J = jaccard_score(ALLANIMESYNOPSIS,POGMOVIESYNOPSIS)
movies_scored_J = pogmovies_by_score(J@score_weights)
movies_scored_J

array([['tt0113957', '32.74696798149941'],
       ['tt0112442', '32.20285129657349'],
       ['tt0338348', '31.87726357632749'],
       ['tt0149261', '31.86336246552418'],
       ['tt1109624', '31.86336246552418'],
       ['tt2404463', '31.86336246552418'],
       ['tt1229238', '31.86336246552418'],
       ['tt0198781', '31.86336246552418'],
       ['tt0052618', '31.86336246552418'],
       ['tt0120660', '31.304691568709597'],
       ['tt0120762', '31.24293127808308'],
       ['tt1187064', '31.24293127808308'],
       ['tt1509767', '31.24293127808308'],
       ['tt1156398', '31.24293127808308'],
       ['tt0245844', '31.24293127808308'],
       ['tt0110357', '30.99158411165053'],
       ['tt0325980', '30.99158411165053'],
       ['tt0120855', '30.99158411165053'],
       ['tt0948470', '30.99158411165053'],
       ['tt0417741', '30.99158411165053'],
       ['tt0312528', '30.948909208093024'],
       ['tt0110763', '30.856151837567868'],
       ['tt0107688', '30.79657216992368'],
       [

In [39]:
C = cosine_score(ALLANIMESYNOPSIS,POGMOVIESYNOPSIS)
movies_scored_C = pogmovies_by_score(C@score_weights)
movies_scored_C

array([['tt1211837', '2.9705039463873546'],
       ['tt1637688', '2.6383208407248646'],
       ['tt0372588', '2.25357925625981'],
       ['tt0980970', '1.9147100706170812'],
       ['tt0112384', '1.8893517508858924'],
       ['tt0106611', '1.8226509774586992'],
       ['tt0177789', '1.8195171060719668'],
       ['tt0257076', '1.7974920446222302'],
       ['tt0770828', '1.7730734387597116'],
       ['tt0376994', '1.761135781078494'],
       ['tt0107048', '1.7532765056653925'],
       ['tt0120685', '1.6960076576588607'],
       ['tt0371724', '1.6729007089139698'],
       ['tt1253863', '1.6703376065419107'],
       ['tt0800080', '1.6626010892030523'],
       ['tt0499448', '1.6530156255103043'],
       ['tt1340138', '1.6455413920034123'],
       ['tt0413267', '1.6348958814892363'],
       ['tt0938283', '1.6269903436559796'],
       ['tt0245429', '1.6107986833448313'],
       ['tt1060277', '1.5991318524938352'],
       ['tt1078912', '1.5915662176181022'],
       ['tt0803096', '1.57658231211

DOCTOR STRANGE. Ok tbh it's kinda hit or miss, probably won't happen on a different run BUT IT'S SOMETHING

Ok so, what if we biased anime where the fire user is the main character compared to anime where the fire user was a secondary or side character?

In [40]:
mc_bias = 1.5

ANIMESYNOPSIS,POGMOVIESYNOPSIS = np.meshgrid(np.concatenate((np.array(list(fire_mc_synopsis_full.values()))[:,0],np.array(list(fire_mcnt_synopsis_full.values()))[:,0])),pogmoviesample['plot_synopsis'])
score_weights = np.concatenate((mc_bias*np.ones(len(fire_mc_synopsis_full)),np.ones(len(fire_mcnt_synopsis_full))))

In [41]:
J = jaccard_score(ANIMESYNOPSIS,POGMOVIESYNOPSIS)
movies_scored_J = pogmovies_by_score(J@score_weights)
movies_scored_J

array([['tt0113957', '171.02292110582226'],
       ['tt0112442', '170.0943848476072'],
       ['tt0149261', '166.71234629634034'],
       ['tt1109624', '166.71234629634034'],
       ['tt2404463', '166.71234629634034'],
       ['tt1229238', '166.71234629634034'],
       ['tt0198781', '166.71234629634034'],
       ['tt0052618', '166.71234629634034'],
       ['tt0338348', '166.30582916362516'],
       ['tt0312528', '165.59763792126483'],
       ['tt0120762', '163.6347059901673'],
       ['tt1187064', '163.6347059901673'],
       ['tt1509767', '163.6347059901673'],
       ['tt1156398', '163.6347059901673'],
       ['tt0245844', '163.6347059901673'],
       ['tt0120660', '162.94764813183338'],
       ['tt2193021', '162.27186965708012'],
       ['tt0110357', '162.1628538365493'],
       ['tt0325980', '162.1628538365493'],
       ['tt0120855', '162.1628538365493'],
       ['tt0948470', '162.1628538365493'],
       ['tt0417741', '162.1628538365493'],
       ['tt0230030', '161.9764274031738'],


In [42]:
C = cosine_score(ANIMESYNOPSIS,POGMOVIESYNOPSIS)
movies_scored_C = pogmovies_by_score(C@score_weights)
movies_scored_C

array([['tt1637688', '11.055994277945704'],
       ['tt1211837', '10.736535872933207'],
       ['tt0372588', '9.514008775340773'],
       ['tt0106611', '9.261425253222061'],
       ['tt0770828', '8.352513158947746'],
       ['tt0980970', '8.29195819122782'],
       ['tt0376994', '8.262726081731692'],
       ['tt0112384', '8.201006317937921'],
       ['tt0177789', '8.071147618052269'],
       ['tt0371724', '8.03648402666192'],
       ['tt0107048', '7.940642776085189'],
       ['tt0257076', '7.786193585793397'],
       ['tt0499448', '7.526245335640416'],
       ['tt0120685', '7.463508478992031'],
       ['tt0800080', '7.374994621875202'],
       ['tt0413267', '7.3544579212546175'],
       ['tt1483013', '7.314377060885988'],
       ['tt2527336', '7.176807479184188'],
       ['tt2109248', '7.092115204624806'],
       ['tt0076759', '7.065994140881842'],
       ['tt0938283', '7.065176627025848'],
       ['tt1060277', '6.921629393333513'],
       ['tt1253863', '6.864458776840342'],
       ['t

Doesn't help... The penalty thing worked tho!